In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from trl import SFTTrainer, KTOTrainer, KTOConfig
from datasets import load_dataset, Dataset, DatasetDict
import os
import torch

/home/mark/.pyenv/versions/mambaforge/envs/cuda_fine_tuning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# datasets = load_dataset('json', data_files="/home/mark/Programming/mosaic-voice/ai_ml/cuda_training/data/processed_dataset_full_KTO.jsonl")
dataset = load_dataset("trl-lib/kto-mix-14k")
# datasets
# train_test_valid = datasets['train'].train_test_split(test_size=0.2)
# # Split the 10% test + valid in half test, half valid
# test_valid = train_test_valid['test'].train_test_split(test_size=0.5)

# dataset = DatasetDict({
#     'train': train_test_valid['train'],
#     'test': test_valid['test'],
#     'validation': test_valid['train']
#     })

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'label'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['prompt', 'completion', 'label'],
        num_rows: 1500
    })
})


In [7]:
def formatting_func_kto(sample, tokenizer, prompt_format):
    # prompt = [
    #     {"content": f"""Assist a non-verbal autistic individual in communicating their thoughts or needs through selected images. \
    #     Your task: infer and articulate the message in first-person, using simple, direct language with empathy and clarity. \
    #     - Be empathetic and direct. - Look for deeper meanings in the input. - Keep the tone practical and straightforward." + {sample["prompt"]}"""}
    # ]

    # completion = [
    #     {"content": sample["completion"]}
    # ]


    # sample["prompt"] = tokenizer.apply_chat_template(prompt, tokenize=False)
    # sample["completion"] = tokenizer.apply_chat_template(completion, tokenize=False)
    
    sample["prompt"] = tokenizer.apply_chat_template(sample["prompt"], tokenize=False)
    sample["completion"] = tokenizer.apply_chat_template(sample["completion"], tokenize=False)

    return sample

In [8]:
def sanity_check(dataset, tokenizer, training_type="SFT", num_samples=3):
    # Sanity check on text dataset with prompts
    for i in range(num_samples):  # Adjust the range to inspect more examples
        print(f"Example {i}:")
        if training_type != "KTO":
            print("Prompt:", dataset['train'][i]['text'])
        else:
            print("Prompt:", dataset['train'][i]['prompt'])
            print("Completion:", dataset['train'][i]['completion'])
            print("Label:", dataset['train'][i]['label'])
        print()

    # Sanity check on tokenized dataset with prompts
    for i in range(num_samples):  # Adjust the range to inspect more examples
        print(f"Example {i}:")
        if training_type != "KTO":
            print("Prompt:", tokenizer.encode(dataset['train'][i]['text']))
        else:
            print("Prompt:", tokenizer.encode(dataset['train'][i]['prompt']))
            print("Completion:", tokenizer.encode(dataset['train'][i]['completion']))
            print("Label:", dataset['train'][i]['label'])
        print()

    # quick sanity check
    print(f'tokenizer.bos_token: {tokenizer.bos_token}')
    print(f'tokenizer.bos_token_id: {tokenizer.bos_token_id}')
    print(f'tokenizer.pad_token: {tokenizer.pad_token}')
    print(f'tokenizer.pad_token_id: {tokenizer.pad_token_id}')
    print(f'tokenizer.eos_token: {tokenizer.eos_token}')
    print(f'tokenizer.eos_token_id: {tokenizer.eos_token_id}')

In [9]:
def format_dataset(model_id, dataset, prompt_format="mistral", training_type="SFT", anton=False):
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, add_bos_token=True)

    if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.unk_token

    if prompt_format == "mistral":
        tokenizer.padding_side = "left"
    formatted_dataset = dataset.map(formatting_func_kto,
                                    batched=False,
                                    fn_kwargs={"prompt_format": prompt_format, "tokenizer": tokenizer})

    print(formatted_dataset)
    sanity_check(formatted_dataset, tokenizer, training_type=training_type, num_samples=3)

    return tokenizer, formatted_dataset

In [11]:
tokenizer, formatted_dataset = format_dataset('mistralai/Mistral-7B-Instruct-v0.2', dataset, 'mistral', 'KTO', False)

Map:   0%|          | 0/13500 [00:00<?, ? examples/s]

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...